In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch


In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head()

,id,post_id,user_id,text,date,spam
0,448401,28850,5.547276e+09,"Ребят, как настроение? Что думаете по поводу п...",2025-03-30 17:32:25,0
1,448406,28850,2.070985e+09,Точно как я сам не догадался,2025-03-30 17:32:49,0
2,448412,28850,6.500166e+09,"из разряда ""пусть все ваши тревоги уносят в ле...",2025-03-30 17:33:12,0
3,448413,28850,5.511331e+09,real.😞,2025-03-30 17:33:27,0
4,448418,28850,7.034223e+08,Чушь) Что там думать),2025-03-30 17:33:46,0


In [6]:
data['spam'].value_counts()

spam
0    6044
1      44
Name: count, dtype: int64

In [13]:
data[data['spam'] == 1].text

90      Вот бы мне написал настоящий, платёжеспособный...
206     ‼️ требуются модели на перманентный макияж ‼️\...
218     3дравствуйте❤\nпомогите,пожалуйста накопить на...
230     ЧТОБЫ ТЕБЯ ПЕТУХА РУБИТЬ ТАКИМ МЕЧОМ! 🔪🖤 Я ТОЛ...
540     🌸💟девоч𝘬u, сдeлаю 𝘱ас𝘬л𝘢д нa т𝘢po💓💗р𝘢з6еpy м𝘢т...
1140    Составлю персональный гороскоп всего за 500 ру...
1204    🌸💟девоч𝘬u, сдeлаю 𝘱ас𝘬л𝘢д нa т𝘢po💓💗р𝘢з6еpy м𝘢т...
1342                                🔥Шkoльнuцы в прoфилe🔥
1343                                🔥Шkoльнuцы в прoфилe🔥
1385    Не cмотpя на тᴏ чтo я девушка, yжᴇ 7 мᴇcяцев н...
1386    Вpᴀг опять попеp в атᴀку! Tyт ʜастᴏящᴀя ᴍясᴏру...
1387    Hе cᴍотᴘя на то что я девyшᴋа, yже 7 месяцeʙ ʜ...
1388    💋 Хочешь отдохнуть от работы и от суеты, но не...
1415                                                 В лс
1627    ❗️ Ищeм пapтнёpoв в зaкpытoe кoмьюнити\n\n🚀Xoч...
1650    Если кто-то задумывался о смене формата или об...
1651             Оставила для тебя только самое вкусное))
1652    Есть т

Синтетика

In [ ]:
post_id_range = (20000, 19000000)
user_id_range = (5e8, 7e9)
start_date = datetime(2025, 3, 15)
end_date = datetime(2025, 4, 30)

synthetic_texts = [
    "💸 ТЫ еще не в деле? Подними 7k за вечер, пишем 👉 @fast_bucks_bot",
    "девочкu, гαдαниe нα Тαρо + мeдиταции 🌌 тoлько ceгoдня",
    "🔥 Шк0льницы ищут зαбoτy… в пρофиле всё нαйдeшь 🔞",
    "Пepeшлu cсылκy 3 друзьям — пoлyчи 1000₽ в пoдaрoк 💰",
    "вcё, чт0 ты xoтел узнать o ceбe — в oднoм paсклaде 🃏",
    "Хочешь узнαть, чтo дeлaют дeвyшkи нoчью? смотри тут 👉 @wetnight",
    "Пpoклaдка пeρвaя — 5к в чac, тoлькo бeз пαнuки 😇",
    "Приветик, я астропсихолог 🌙 сделаю твой путь светлее 💫 пиши в ЛС",
    "тeбя пpοклuнyли... yспей снять зa 499₽ сегодня! 🧿",
    "🚨 экстренная рассылка! вход в клуб инвесторов @real_crypto_vip",
    "🤯 Ты не поверишь, что она делает... @dark_secret",
    "Пpосто скaжи ДA – и твоя жизнь пοмeняется: @yes_miracles",
    "Заработок на ставках ⚽️ от 15k в неделю! @pro_bets",
    "тoлъко ceйчac! твoя любимая девчoнκa ждeτ в лc 😈",
    "чекни мoй профиль — там всё, чeгo ты искаɦ 😇",
    "💥 Группa закрыта! oстaвь зaявκy на дοступ: @αlpha_team",
    "cκopо зaкpытиe набора... yспей войти в тему 💼",
    "🥷 Самые тихие деньги делаются без шума. Смотри в bio.",
    "Сегодня луна в Деве 🌕 идеальное время для очищения 💫 пиши",
    "🎲 Хочешь пассив? Бoт capит 2к в день: @ez_passive",
    "ДA ты coздaн для этогo! Прocτo зайди и узнай: @true_way",
    "мaмa гoвopила, я ocтaнyсь бeднoй… a теперь я в топе Forbes",
    "Знaκoмcтβα, кoтopыe зaκaнчивaютcя нeжнocтью… в лс 💌",
    "🚨 Срочный отлuв babкu нaкaнyнe блoкuрoвки! @cash_drain",
    "тeбe пoнραвится мoй coн... в λс oпиcaнuе 💭",
    "xαчу пοдaρить тeбe εмoцuu. oткρой ссылκу 💖 https://bit.ly/fakegift",
    "💬 зaдaй вoπρoc, пoлyчи иcτину. oнлaйн oракyл гοтов",
    "я бывший вοeнный, τепepь живу пo-дpyгому – нaпиши, paсcκaжу",
    "не ищи смысл — он тут 👉 @real_zen_life",
    "Пpямoй вxод в кpиптoтeму, покa бeз καпиταлa: @crypto_hole",
    "мы бyдем дeлать eтo всю ночь… пока не уснёшь 😈💤",
    "🌸 чaкρы зaбиты? y мeня eсть ритуaл, кoτopый пοмoжeт",
    "cκopο вce yйдeт в минyc, a мы yже в плюсe 💹",
    "зαглянu в пροфиль — ecли тeбe нe cтpaшнo 😏",
    "💄 oнa xoчет быть нeжнοй, нο зaeдeт тeбя дo слёз",
    "этo не пpoстo кoммент — этo знaк 💫",
    "кaκ ты мнe нραвишьcя… xoчу зακpытoe – в директ 💕",
    "https://bit.ly/lucky_4_you — твoй кαпκaн yже oткрылcя 🎯",
    "καждый дeнь c тaкой, кaκ я — этo праздник 🥂",
    "Нe тяни… Пиши «+» в ЛС и получи доступ 🗝️",
    "я тeбя зaкaжу, как чай лунный — нa ночь и в грезы 🌕",
    "пpoклятие снятo, но я oстaнyсь в тeбe навceгдa 💀",
    "💬 нe вceгдa лeгκо быть мαгoм, нo этo peαльнo",
    "в 17 я был никем, в 21 — y мeня лaмбa 💎",
    "нyжeн ли тeбe eщe шaнc? он в пρофuлe…",
    "Στραннo, нο мнe нравишься. пиши, и yзнaeшь пoчeмy",
    "Cлyчайнo? нe думaю. Πepейдu пo ccылκe @my_f8_bot",
    "мoи δυмы тeбe — пoдарκом. Пpoчтu в директе 💌",
    "каждое утро – c нoвoй cκaзκой. Твоя здеcь 👉 @𝘥𝘳𝘦𝘢𝘮𝘪𝘯𝘨",
    "🌪️Тyт вce для кайфa: мeф, шиш, αльфα⚡ Пиши 👉 @stormhigh",
    "💊 сk👁️p в наличии. Kуpтkи пo лyчшeй цeнe! пиши: @cloudtripz",
    "α❄️льфa, мeф, гaш 🍭 — oтправкa 24/7 💬 @deepstuff",
    "🧃сκ в таблax — тёплые руки ждyт твой зaк🌫️ t[.]me/tabdream",
    "🍬Мaлuнoвый мeф, кaк в cказке🎠, пpямoй кaнaл: @smoke_magic",
    "ШиШки 🚬 кубы 🧊 и кислa 💣 — лeтuм! напиши @fastlane420",
    "Лcд, ск и a-пвp 🐉 тoлькo oт \"добpaкa\" ✅ t[.]me/d0bra_vech",
    "Хoчeшь рeaльный шмaль? 🌿 гaш, блэк и xимкa! @🌙moonkush",
    "Пcилоцибинoвые 🍄 capы, лcд 💡, пeйoт ⛺ @cactus_trip",
    "𝘮3ф, кoнфeтkи, лcд — oпт и poзницa 💼 Пиши @sweetrave",
    "Aльфα oт вoенки 🚁, без шлaкa, только плaвнoe ухoдoвoe 🎧 — @darknightpush",
    "Bзял y ниx куб, пoлeтeл 💎 t[.]me/kubtiger 🧠",
    "👽👁️ черeз 7 мин y тeбя мeф в рукe — @n0rmaltrip",
    "💉 Пepвaя дoстaвкa — бecплaтнo! Шиш, 𝓪-льфa, сκ 💬 t[.]me/sk_rabbit",
    "Aльфа 🌪️ a-пвp ☄️ ск 💥 Вeчepниe пocтвaвки 19:00–02:00 — @night_fix",
    "🌈 3д куб, лeгaл, cк и плюxa — @dr33my_drop в кoнтaкт!",
    "шишки от брата 😎🔥 куришь и в мyлтик влетаешь t[.]me/treewave",
    "🎯 Мeфaмaт в мeтpoвых клaдкax 💬 пишu: @eztripx",
    "oлд-скyл ЛСД 🌀 с eфeктом мeдoвoгo удаpа 🍯 — @vintagehits",
    "Xимия, кaк нa 2k15 💿 и шиш, как в НЛО! 👾 @retropack420"
]

while len(synthetic_texts) < 130:
    synthetic_texts.append(random.choice(synthetic_texts) + " ✨")

synthetic_data = pd.DataFrame({
    "id": list(range(9000000, 9000000 + len(synthetic_texts))),
    "post_id": [random.randint(*post_id_range) for _ in range(len(synthetic_texts))],
    "user_id": [random.uniform(*user_id_range) for _ in range(len(synthetic_texts))],
    "text": synthetic_texts,
    "date": [start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds()))) for _ in range(len(synthetic_texts))],
    "spam": [1] * len(synthetic_texts)
})

synthetic_data.head()

,id,post_id,user_id,text,date,spam
0,9000000,5078913,4.452523e+09,"💸 ТЫ еще не в деле? Подними 7k за вечер, пишем...",2025-04-18 17:11:26,1
1,9000001,16103186,1.016865e+09,"девочкu, гαдαниe нα Тαρо + мeдиταции 🌌 тoлько ...",2025-04-10 06:46:52,1
2,9000002,10698936,5.952706e+09,🔥 Шк0льницы ищут зαбoτy… в пρофиле всё нαйдeшь 🔞,2025-04-11 03:13:32,1
3,9000003,12106690,6.425954e+09,Пepeшлu cсылκy 3 друзьям — пoлyчи 1000₽ в пoдa...,2025-03-28 04:35:44,1
4,9000004,4101837,2.238063e+09,"вcё, чт0 ты xoтел узнать o ceбe — в oднoм paск...",2025-03-15 19:42:39,1


In [20]:
full_data = pd.concat([data, synthetic_data], ignore_index=True)
full_data

,id,post_id,user_id,text,date,spam
0,448401,28850,5.547276e+09,"Ребят, как настроение? Что думаете по поводу п...",2025-03-30 17:32:25,0
1,448406,28850,2.070985e+09,Точно как я сам не догадался,2025-03-30 17:32:49,0
2,448412,28850,6.500166e+09,"из разряда ""пусть все ваши тревоги уносят в ле...",2025-03-30 17:33:12,0
3,448413,28850,5.511331e+09,real.😞,2025-03-30 17:33:27,0
4,448418,28850,7.034223e+08,Чушь) Что там думать),2025-03-30 17:33:46,0
...,...,...,...,...,...,...
6213,9000125,7625401,3.101168e+09,мoи δυмы тeбe — пoдарκом. Пpoчтu в директе 💌 ✨,2025-03-30 23:24:00,1
6214,9000126,13610190,6.908491e+09,👽👁️ черeз 7 мин y тeбя мeф в рукe — @n0rmaltrip ✨,2025-04-12 07:46:37,1
6215,9000127,7194060,2.934258e+09,"девочкu, гαдαниe нα Тαρо + мeдиταции 🌌 тoлько ...",2025-04-26 05:29:26,1
6216,9000128,6303540,4.716226e+09,oлд-скyл ЛСД 🌀 с eфeктом мeдoвoгo удаpа 🍯 — @v...,2025-03-29 23:58:27,1


In [ ]:
real_data = full_data[full_data['spam'] != 1]  

train_real, temp_real = train_test_split(real_data, test_size=0.2, random_state=42, stratify=real_data['spam'])
val_real, test_real = train_test_split(temp_real, test_size=0.5, random_state=42, stratify=temp_real['spam'])
train_final = pd.concat([train_real, synthetic_data], ignore_index=True)

In [23]:
print("Train set:\n", train_final['spam'].value_counts())
print("Validation set:\n", val_real['spam'].value_counts())
print("Test set:\n", test_real['spam'].value_counts())

Train set:
 spam
0    4835
1     130
Name: count, dtype: int64
Validation set:
 spam
0    604
Name: count, dtype: int64
Test set:
 spam
0    605
Name: count, dtype: int64


In [ ]:

synthetic_val = synthetic_data.sample(15, random_state=42).copy()
synthetic_test = synthetic_data.drop(synthetic_val.index).sample(15, random_state=43).copy()

synthetic_val["source"] = "synthetic_val"
synthetic_test["source"] = "synthetic_test"

val_real["source"] = "real"
test_real["source"] = "real"

val_augmented = pd.concat([val_real, synthetic_val], ignore_index=True).sample(frac=1, random_state=44)
test_augmented = pd.concat([test_real, synthetic_test], ignore_index=True).sample(frac=1, random_state=45)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train = tfidf.fit_transform(train_final['text'])
X_val = tfidf.transform(val_real['text'])
X_test = tfidf.transform(test_real['text'])

y_train = train_final['spam']
y_val = val_real['spam']
y_test = test_real['spam']

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

model = LogisticRegression(max_iter=1000, solver='liblinear')  # или 'saga'
model.fit(X_train_smote, y_train_smote)

print("📊 Validation set:")
y_val_pred = model.predict(X_val)
print(classification_report(y_val, y_val_pred, digits=4))

print("📊 Test set:")
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred, digits=4))

print("Confusion matrix (test):")
print(confusion_matrix(y_test, y_test_pred))


📊 Validation set:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       604

    accuracy                         1.0000       604
   macro avg     1.0000    1.0000    1.0000       604
weighted avg     1.0000    1.0000    1.0000       604

📊 Test set:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       605

    accuracy                         1.0000       605
   macro avg     1.0000    1.0000    1.0000       605
weighted avg     1.0000    1.0000    1.0000       605

Confusion matrix (test):
[[605]]


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
X_val = tfidf.transform(val_augmented['text'])
y_val = val_augmented['spam']
X_test = tfidf.transform(test_augmented['text'])
y_test = test_augmented['spam']

y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

print("Validation set (augmented):")
print(classification_report(y_val, y_val_pred, digits=4))

print("Test set (augmented):")
print(classification_report(y_test, y_test_pred, digits=4))

val_probs = model.predict_proba(X_val)[:, 1]
test_probs = model.predict_proba(X_test)[:, 1]

print("ROC AUC (val):", roc_auc_score(y_val, val_probs))
print("PR AUC (val):", average_precision_score(y_val, val_probs))

print("ROC AUC (test):", roc_auc_score(y_test, test_probs))
print("PR AUC (test):", average_precision_score(y_test, test_probs))


📊 Validation set (augmented):
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       604
           1     1.0000    1.0000    1.0000        15

    accuracy                         1.0000       619
   macro avg     1.0000    1.0000    1.0000       619
weighted avg     1.0000    1.0000    1.0000       619

📊 Test set (augmented):
              precision    recall  f1-score   support

           0     0.9983    1.0000    0.9992       605
           1     1.0000    0.9333    0.9655        15

    accuracy                         0.9984       620
   macro avg     0.9992    0.9667    0.9823       620
weighted avg     0.9984    0.9984    0.9984       620

ROC AUC (val): 1.0
PR AUC (val): 0.9999999999999999
ROC AUC (test): 0.9959228650137741
PR AUC (test): 0.9459915611814345


In [ ]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)

y_val_pred_rf = rf_model.predict(X_val)
print("Random Forest - Validation Report:")
print(classification_report(y_val, y_val_pred_rf))

y_test_pred_rf = rf_model.predict(X_test)
print("Random Forest - Test Report:")
print(classification_report(y_test, y_test_pred_rf))


Random Forest - Validation Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       604
           1       1.00      1.00      1.00        15

    accuracy                           1.00       619
   macro avg       1.00      1.00      1.00       619
weighted avg       1.00      1.00      1.00       619

Random Forest - Test Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       605
           1       1.00      0.93      0.97        15

    accuracy                           1.00       620
   macro avg       1.00      0.97      0.98       620
weighted avg       1.00      1.00      1.00       620



In [ ]:
balance_ratio = len(y_train) / (2 * sum(y_train))

xgb_model = xgb.XGBClassifier(scale_pos_weight=balance_ratio, random_state=42)
xgb_model.fit(X_train, y_train)

y_val_pred_xgb = xgb_model.predict(X_val)
print("XGBoost - Validation Report:")
print(classification_report(y_val, y_val_pred_xgb))

y_test_pred_xgb = xgb_model.predict(X_test)
print("XGBoost - Test Report:")
print(classification_report(y_test, y_test_pred_xgb))


XGBoost - Validation Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       604
           1       1.00      1.00      1.00        15

    accuracy                           1.00       619
   macro avg       1.00      1.00      1.00       619
weighted avg       1.00      1.00      1.00       619

XGBoost - Test Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       605
           1       1.00      0.93      0.97        15

    accuracy                           1.00       620
   macro avg       1.00      0.97      0.98       620
weighted avg       1.00      1.00      1.00       620



1. LightGBM

In [ ]:
lgb_model = lgb.LGBMClassifier(class_weight='balanced', random_state=42)
lgb_model.fit(X_train, y_train)

y_val_pred_lgb = lgb_model.predict_proba(X_val)[:, 1]
y_test_pred_lgb = lgb_model.predict_proba(X_test)[:, 1]

roc_auc_lgb_val = roc_auc_score(y_val, y_val_pred_lgb)
roc_auc_lgb_test = roc_auc_score(y_test, y_test_pred_lgb)
print(f"ROC AUC for LightGBM (Validation): {roc_auc_lgb_val}")
print(f"ROC AUC for LightGBM (Test): {roc_auc_lgb_test}")


[LightGBM] [Info] Number of positive: 130, number of negative: 4835
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008451 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5354
[LightGBM] [Info] Number of data points in the train set: 4965, number of used features: 225
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [ ]:
catboost_model = CatBoostClassifier(class_weights=[1, len(y_train) / sum(y_train)], random_state=42, iterations=500, learning_rate=0.1, depth=10, silent=True)
catboost_model.fit(X_train, y_train)

y_val_pred_catboost = catboost_model.predict_proba(X_val)[:, 1]
y_test_pred_catboost = catboost_model.predict_proba(X_test)[:, 1]

roc_auc_catboost_val = roc_auc_score(y_val, y_val_pred_catboost)
roc_auc_catboost_test = roc_auc_score(y_test, y_test_pred_catboost)
print(f"ROC AUC for CatBoost (Validation): {roc_auc_catboost_val}")
print(f"ROC AUC for CatBoost (Test): {roc_auc_catboost_test}")


ROC AUC for CatBoost (Validation): 1.0
ROC AUC for CatBoost (Test): 0.9923966942148761


In [ ]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
xgb_model = xgb.XGBClassifier(scale_pos_weight=balance_ratio, random_state=42)
lgb_model = lgb.LGBMClassifier(class_weight='balanced', random_state=42)

ensemble_model = VotingClassifier(estimators=[('rf', rf_model), ('xgb', xgb_model), ('lgb', lgb_model)], voting='soft')
ensemble_model.fit(X_train, y_train)

y_val_pred_ensemble = ensemble_model.predict_proba(X_val)[:, 1]
y_test_pred_ensemble = ensemble_model.predict_proba(X_test)[:, 1]

roc_auc_ensemble_val = roc_auc_score(y_val, y_val_pred_ensemble)
roc_auc_ensemble_test = roc_auc_score(y_test, y_test_pred_ensemble)
print(f"ROC AUC for Ensemble Model (Validation): {roc_auc_ensemble_val}")
print(f"ROC AUC for Ensemble Model (Test): {roc_auc_ensemble_test}")


[LightGBM] [Info] Number of positive: 130, number of negative: 4835
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003395 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5354
[LightGBM] [Info] Number of data points in the train set: 4965, number of used features: 225
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 